In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os

import numpy as np
import pandas as pd
from astropy.table import Table
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

from agnfinder.prospector import main, load_photometry
from agnfinder import photometry_to_table

from sklearn.covariance import GraphicalLasso, GraphicalLassoCV, MinCovDet, EmpiricalCovariance


In [ ]:
assert False  # deprecated notebook

Simplest thing that works: use the mean uncertainty at that magnitude

In [141]:
from agnfinder.simulation_samples import get_photometry_simulator

In [ ]:
simulator = get_photometry_simulator(emulate_ssp=False)

In [ ]:
# X = np.random.rand(18,28)
# X[12, 12] = 100
# cov = MinCovDet().fit(X)
# cov.covariance_.shape

In [ ]:
# np.around(cov.covariance_, decimals=2)

In [ ]:

from sklearn.preprocessing import normalize

In [ ]:
X = reliable_df.dropna().values
X.shape
assert not np.isnan(X).any()

In [ ]:
cov = GraphicalLassoCV(max_iter=1000, cv=5).fit(X)
# cov = EmpiricalCovariance().fit(X)
# np.around(cov.covariance_, decimals=3)

In [ ]:
labelled_cov = pd.DataFrame(cov.covariance_, index=reliable_df.columns.values, columns=reliable_df.columns.values)
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(labelled_cov, ax=ax)

In [ ]:
inv_cov = np.linalg.inv(cov.covariance_)

In [ ]:
labelled_cov = pd.DataFrame(inv_cov, index=reliable_df.columns.values, columns=reliable_df.columns.values)
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(labelled_cov, ax=ax)

In [ ]:
log_X = np.log10(X)
# X = normalize(np.log10(X))

In [ ]:
log_cov = GraphicalLassoCV(max_iter=10000, cv=5).fit(log_X)

In [ ]:
labelled_log_cov = pd.DataFrame(log_cov.covariance_, index=reliable_df.columns.values, columns=reliable_df.columns.values)
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(labelled_log_cov, ax=ax)

In [ ]:
inv_log_cov = np.linalg.inv(log_cov.covariance_)

In [ ]:
np.around(inv_log_cov, decimals=3)

In [ ]:
labelled_log_cov = pd.DataFrame(inv_log_cov, index=reliable_df.columns.values, columns=reliable_df.columns.values)
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(labelled_log_cov, ax=ax)

In [ ]:
from scipy.stats import multivariate_normal

In [ ]:
log_norm = multivariate_normal(mean=None, cov=log_cov.covariance_)
norm = multivariate_normal(mean=None, cov=cov.covariance_)

In [ ]:
fig, ax = plt.subplots()
for i in range(100):
    ax.scatter(range(12), norm.rvs(), alpha=0.2)

In [ ]:
plt.loglog(range(12), np.abs(norm.rvs()), 'x')
# plt.ylim([-2., 2.])

In [ ]:
np.log10(np.abs(norm.rvs()))

In [ ]:
plt.scatter(range(12), log_norm.rvs())
plt.ylim([-2., 2.])